In [1]:
import skimage as si
from scipy import misc
import numpy as np
import math

In [2]:
rgb_image_original = misc.imread("kaktus256.bmp")

# prosirivanje slike ukoliko njena visina ili sirina nije deljiva sa 8
# da bi mogla da se podeli na 8x8 blokove
sirina_originalne = rgb_image_original.shape[0]
visina_originalne = rgb_image_original.shape[1]

if sirina_originalne % 8 != 0:
    nova_sirina = sirina_originalne + (8 - (sirina_originalne % 8))
else:
    nova_sirina = sirina_originalne

if visina_originalne % 8 != 0:
    nova_visina = visina_originalne + (8 - (sirina_originalne % 8))
else:
    nova_visina = visina_originalne
    
rgb_image = np.zeros((nova_sirina, nova_visina, 3), dtype='int16')
rgb_image[0:sirina_originalne,0:visina_originalne,0] = rgb_image_original[:,:,0]
rgb_image[0:sirina_originalne,0:visina_originalne,1] = rgb_image_original[:,:,1]
rgb_image[0:sirina_originalne,0:visina_originalne,2] = rgb_image_original[:,:,2]


# izdvajanje matrice za svaku boju posebno
crvena = np.array(rgb_image[:,:,0],dtype='int16')
zelena = np.array(rgb_image[:,:,1],dtype='int16')
plava = np.array(rgb_image[:,:,2],dtype='int16')

# siva = np.array((crvena+zelena+plava)//3,dtype='int16')

In [3]:
# rgb_image[:,:,0] = siva
# rgb_image[:,:,1] = siva
# rgb_image[:,:,2] = siva

In [4]:
# misc.imsave("gray_image.bmp",rgb_image)

In [5]:
# izracunavanje broja blokova 8x8
broj_vertikalnih_blokova = rgb_image.shape[0] // 8
broj_horizontalnih_blokova = rgb_image.shape[1] // 8

In [6]:
# pravljenje matrice diskretne kosinusne transformacije (DCT)
T = np.zeros((8, 8))
T[0] = 1/(math.sqrt(8))
for i in range(1, 8):
    for j in range(0, 8):
        T[i][j] = math.sqrt(2/8)*math.cos(((2*j+1)*i*math.pi)/(2*8))

In [7]:
# pravljenje kvantizacione matrice
Q50 = np.array([[16, 11, 10, 16, 24, 40, 51, 61],
              [12, 12, 14, 19, 26, 58, 60, 55],
              [14, 13, 16, 24, 40, 57, 69, 56],
              [14, 17, 22, 29, 51, 87, 80, 62],
              [18, 22, 37, 56, 68, 109, 103, 77],
              [24, 35, 55, 64, 81, 104, 113, 92],
              [49, 64, 78, 87, 103, 121, 120, 101],
              [72, 92, 95, 98, 112, 100, 103, 99]],dtype='int16')

# na osnovu izbora nivoa kvaliteta pravi se nova kvantizaciona matrica
nivo_kvaliteta = 50

# nivo kvaliteta treba da bude izmedju 1 i 99
if nivo_kvaliteta < 1:
    nivo_kvaliteta = 1
if nivo_kvaliteta > 99:
    nivo_kvaliteta = 100

# racunanje nove kvantizacione matrice za izabrani kvalitet
if nivo_kvaliteta > 50:
    Q = (100-nivo_kvaliteta)/50*Q50
else:
    if nivo_kvaliteta < 50:
        Q = 50/nivo_kvaliteta*Q50
    else:
        Q = Q50

Q = np.array(Q, dtype='int16')    

# provera da boja nije izasla iz predvidjenog okvira 0-255
for i in range(0, 8):
    for j in range(0, 8):
        if Q[i, j] > 255:
            Q[i, j] = 255
        if Q[i, j] < 1:
            Q[i, j] = 1

In [8]:
# kompresija 

# inicijalizacija matrica
kompresovana_crvena = np.zeros(rgb_image.shape[0:2])
kompresovana_zelena = np.zeros(rgb_image.shape[0:2])
kompresovana_plava = np.zeros(rgb_image.shape[0:2])

# inicijalizacija broja nula za svaku boju
broj_nula_crvena = 0
broj_nula_zelena = 0
broj_nula_plava = 0

# za svaki blok 8x8 i svaku boju
for i in range(0, broj_vertikalnih_blokova):
    for j in range(0, broj_horizontalnih_blokova):
        # izdvajanje odgovarajuceg bloka
        original_crvena = crvena[i*8 : (i*8+8), j*8 : (j*8+8)]
        original_zelena = zelena[i*8 : (i*8+8), j*8 : (j*8+8)]
        original_plava = plava[i*8 : (i*8+8), j*8 : (j*8+8)]
        
        # prebacivanje u opseg -128-127 sa kojim radi DCT
        M_crvena = original_crvena-128
        M_zelena = original_zelena-128
        M_plava = original_plava-128
        
        # primena diskretne kosinusne transformacije (DCT)
        D_crvena = T.dot(M_crvena.dot(T.transpose()))
        D_zelena = T.dot(M_zelena.dot(T.transpose()))
        D_plava = T.dot(M_plava.dot(T.transpose()))
        
        # kvantizacija - deljenje kvantizacionom matricom
        C_crvena = np.round(np.divide(D_crvena,Q))
        C_zelena = np.round(np.divide(D_zelena,Q))
        C_plava = np.round(np.divide(D_plava,Q))
        
        # brojenje nula u dobijenoj kompresovanoj matrici
        for m in range(0, 8):
            for n in range(0, 8):
                if C_crvena[m, n] == 0:
                    broj_nula_crvena += 1
                if C_zelena[m, n] == 0:
                    broj_nula_zelena += 1
                if C_plava[m, n] == 0:
                    broj_nula_plava+= 1
                
        # prepisivanje u kompresovane matrice cele slike
        kompresovana_crvena[i*8 : (i*8+8), j*8 : (j*8+8)] = C_crvena
        kompresovana_zelena[i*8 : (i*8+8), j*8 : (j*8+8)] = C_zelena
        kompresovana_plava[i*8 : (i*8+8), j*8 : (j*8+8)] = C_plava

# procenat 0 u matricama:
procenat_nula = (broj_nula_crvena + broj_nula_plava + broj_nula_zelena) / (3 * rgb_image.shape[0] * rgb_image.shape[1]) * 100
print('Procenat nula u matricama je: % .2f ' % procenat_nula)

Procenat nula u matricama je:  86.44 


In [9]:
# dekompresija

# inicijalizacija matrica
dekompresovana_crvena = np.zeros(rgb_image.shape[0:2])
dekompresovana_zelena = np.zeros(rgb_image.shape[0:2])
dekompresovana_plava = np.zeros(rgb_image.shape[0:2])

# za svaki blok 8x8 i svaku boju
for i in range(0, broj_vertikalnih_blokova):
    for j in range(0, broj_horizontalnih_blokova):
        # izdvajanje odgovarajuceg bloka
        C_crvena = kompresovana_crvena[i*8 : (i*8+8), j*8 : (j*8+8)]
        C_zelena = kompresovana_zelena[i*8 : (i*8+8), j*8 : (j*8+8)]
        C_plava = kompresovana_plava[i*8 : (i*8+8), j*8 : (j*8+8)]
        
        # mnozenje kvantizacionom matricom
        R_crvena = np.multiply(C_crvena,Q)
        R_zelena = np.multiply(C_zelena,Q)
        R_plava = np.multiply(C_plava,Q)
        
        # primena inverzne diskretne kosinusne transformacije (IDCT)
        N_crvena = np.round((T.transpose()).dot(R_crvena.dot(T))) + 128
        N_zelena = np.round((T.transpose()).dot(R_zelena.dot(T))) + 128
        N_plava = np.round((T.transpose()).dot(R_plava.dot(T))) + 128
        
        # provera da boja nije izasla iz predvidjenog okvira 0-255
        for m in range(0, 8):
            for n in range(0, 8):
                if N_crvena[m, n] > 255:
                    N_crvena[m, n] = 255
                if N_crvena[m, n] < 0:
                    N_crvena[m, n] = 0
                if N_zelena[m, n] > 255:
                    N_zelena[m,n] = 255
                if N_zelena[m, n] < 0:
                    N_zelena[m, n] = 0
                if N_plava[m, n] > 255:
                    N_plava[m, n] = 255
                if N_plava[m, n] < 0:
                    N_plava[m, n] = 0
                    
        # prepisivanje u dekompresovanu matricu cele slike
        dekompresovana_crvena[i*8 : (i*8+8), j*8 : (j*8+8)] = N_crvena
        dekompresovana_zelena[i*8 : (i*8+8), j*8 : (j*8+8)] = N_zelena
        dekompresovana_plava[i*8 : (i*8+8), j*8 : (j*8+8)] = N_plava

# cuvanje nove slike
rgb_image_original[:,:,0] = dekompresovana_crvena[0:sirina_originalne,0:visina_originalne]
rgb_image_original[:,:,1] = dekompresovana_zelena[0:sirina_originalne,0:visina_originalne]
rgb_image_original[:,:,2] = dekompresovana_plava[0:sirina_originalne,0:visina_originalne]
misc.imsave("gray_image_compressed50.bmp",rgb_image_original)

In [10]:
#primer sa pdf-a (provera)
original1 = np.array([[154, 123, 123, 123, 123, 123, 123, 136],
              [192, 180, 136, 154, 154, 154, 136, 110],
              [254, 198, 154, 154, 180, 154, 123, 123],
              [239, 180, 136, 180, 180, 166, 123, 123],
              [180, 154, 136, 167, 166, 149, 136, 136],
              [128, 136, 123, 136, 154, 180, 198, 154],
              [123, 105, 110, 149, 136, 136, 180, 166],
              [110, 136, 123, 123, 123, 136, 154, 136]])
M1 = original1-128
D1 = T.dot(M1.dot(T.transpose()))
C1 = np.round(np.divide(D1,Q50))
C1

R1 = np.multiply(C1,Q50)
N1 = np.round((T.transpose()).dot(R1.dot(T))) + 128
N1

array([[ 149.,  134.,  119.,  116.,  121.,  126.,  127.,  128.],
       [ 204.,  168.,  140.,  144.,  155.,  150.,  135.,  125.],
       [ 253.,  195.,  155.,  166.,  183.,  165.,  131.,  111.],
       [ 245.,  185.,  148.,  166.,  184.,  160.,  124.,  107.],
       [ 188.,  149.,  132.,  155.,  172.,  159.,  141.,  136.],
       [ 132.,  123.,  125.,  143.,  160.,  166.,  168.,  171.],
       [ 109.,  119.,  126.,  128.,  139.,  158.,  168.,  166.],
       [ 111.,  127.,  127.,  114.,  118.,  141.,  147.,  135.]])